In [1]:
from pok_env_gym_RLLib import PokemonEnv
from ray.tune.registry import register_env


In [2]:
def env_creator(arg):
    print(arg)
    return PokemonEnv()

register_env("PokemonEnv", env_creator)

In [3]:
from ray.rllib.algorithms.dqn.dqn import DQNConfig, DQN
from ray.tune.logger import pretty_print

replay_config = {
        "type": "MultiAgentPrioritizedReplayBuffer",
        "capacity": 60000,
        "prioritized_replay_alpha": 0.5,
        "prioritized_replay_beta": 0.5,
        "prioritized_replay_eps": 3e-6,
    }

config = (DQNConfig()
          .training(replay_buffer_config=replay_config,
                         train_batch_size=64,
                         n_step=1,
                         num_atoms=1,
                         double_q=False,
                         dueling=False,
                         num_steps_sampled_before_learning_starts=100)
            .resources(num_gpus=0)
            .rollouts(num_rollout_workers=2)
            .environment("PokemonEnv"))
algo = DQN(config=config)
for i in range(100):
   print(pretty_print(algo.train())) 


2024-03-07 17:58:09,970	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
/Users/jskaf/miniconda3/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/jskaf/miniconda3/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._logger

(RolloutWorker pid=95588) {, worker=1/2, vector_idx=0, remote=False}


(RolloutWorker pid=95588) 2024-03-07 17:58:14,191	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.


(RolloutWorker pid=95588) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0


2024-03-07 17:58:15,887	WARNING util.py:62 -- Install gputil for GPU system monitoring.
2024-03-07 17:58:16,213	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
(RolloutWorker pid=95588) 2024-03-07 17:58:16.772 python[95588:4402642] Warning: Window move completed without beginning


(RolloutWorker pid=95589) {, worker=2/2, vector_idx=0, remote=False}
(RolloutWorker pid=95589) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0 [repeated 58x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


2024-03-07 17:58:22,844	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


(RolloutWorker pid=95589) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0 [repeated 68x across cluster]
(RolloutWorker pid=95588) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0 [repeated 62x across cluster]
(RolloutWorker pid=95589) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0 [repeated 60x across cluster]
(RolloutWorker pid=95589) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0 [repeated 60x across cluster]
(RolloutWorker pid=95589) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0 [repeated 63x across cluster]
(RolloutWorker pid=95589) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0 [repeated 62x across cluster]
(RolloutWorker pid=95589) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0 [repeated 61x across cluster]
(RolloutWorker pid=95589) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0 [repeated 62x across cluster]
(RolloutWorker pid=95589) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploratio